In [1]:
import pandas as pd
from siuba import *
import numpy as np

In [2]:
import re

In [11]:
# !pip install nltk
# ! pip install textblob
# ! pip install wordcloud

In [7]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

In [8]:
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', 1000)

In [9]:
df= pd.read_parquet("dla_df.parquet")

In [10]:
df.sample()

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,process_days,primary_agency_name,adjusted_total_requested,adjusted_fed_requested,adjusted_ac_requested,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
2071,Obligated,STPL,5193(041),Taft,2018-03-20,2018-03-21,2018-03-22,2018-03-22,2018-04-03,222490.0,0.0,251318.0,Authorized,5193,6,E-76 approved on,5.0,1.0,12.0,KER160402,6th St. Between Center St. And Kern St.,"Reconstruct Pedestrian Facilities, Existing Curb Ramps, Sidewalks, And Alley",1,2018-03-16,2018-03-21,KCOG,5193,2018.0,18.0,Taft,271197.856622,240089.492674,0.0,1,0,0,1,0,1,0,3


In [94]:
"""
from Tiffany's new-metrics notebook
"""

ACTIVE_TRANSPORTATION_2 = ['bike', 'bicycle', 'cyclist', 
                         'pedestrian', 
                         ## including the spelling errors of `pedestrian`
                         'pedestrain',
                         'crosswalk', 
                         'bulb out', 'bulb-out', 
                         'active transp', 'traffic reduction', 
                         'speed reduction', 'ped', 'srts', 
                         'safe routes to school',
                         'sidewalk', 'side walk', 'Cl ', 'trail'
                        ]
TRANSIT_2 = ['bus', 'metro', 'station', #Station comes up a few times as a charging station and also as a train station
           'transit','fare', 'brt', 'yarts', 'rail', 'highway-rail'
           # , 'station' in description and 'charging station' not in description
          ] 
BRIDGE_2 = ["bridge", 'viaduct']
STREET_2 = ['traffic signal', 'resurface', 'resurfacing', 'slurry', 'seal' 
          'sign', 'stripe', 'striping', 'median', 
          'guard rail', 'guardrail', 
          'road', 'street', 
          'sinkhole', 'intersection', 'signal', 'curb',
          'light', 'tree', 'pavement', 'roundabout'
         ]

FREEWAY_2 = ['hov ', 'hot ', 'freeway', 'highway', 'express lanes', 'hwy']


INFRA_RESILIENCY_ER_2 = ['repair', 'emergency', 'replace','retrofit', 'er',
                       'rehab', 'improvements', 'seismic', 'reconstruct', 'restoration']

CONGESTION_RELIEF_2 = ['congestion', 'rideshare','ridesharing', 'vanpool', 'car share', 'demand management']

NOT_INC_2 = ['charging', 'fueling', 'cng', 'bridge', 'trail',
           'k-rail', 'guardrails', 'bridge rail', 'guard', 'guarrail'
          ]

def categorize_project_descriptions_2(row):
    """
    This function takes a individual type of work description (row of a dataframe)
    and returns a dummy flag of 1 if it finds keyword present in
    project categories (active transportation, transit, bridge, etc).
    A description can contain multiple keywords across categories.
    """
    # Make lowercase
    description = row.type_of_work.lower()
    
    # Store a bunch of columns that will be flagged
    # A project can involve multiple things...also, not sure what's in the descriptions
    active_transp_2 = 0
    transit_2 = 0
    bridge_2 = 0
    street_2 = 0
    freeway_2 = 0
    infra_resiliency_er_2 = 0
    congestion_relief_2 = 0
        
    if any(word in description for word in ACTIVE_TRANSPORTATION):
        active_transp_2 = 1
        
    #if any(word in description if instanceof(word, str) else word(description) for word in TRANSIT)

    if (any(word in description for word in TRANSIT_2) and 
        not any(exclude_word in description for exclude_word in NOT_INC_2)
       ):
        transit_2 = 1
        
    if any(word in description for word in BRIDGE_2):
        bridge_2 = 1
    if any(word in description for word in STREET_2):
        street_2 = 1
    if any(word in description for word in FREEWAY_2):
        freeway_2 = 1 
    if any(word in description for word in INFRA_RESILIENCY_ER_2):
        infra_resiliency_er_2 = 1
    if any(word in description for word in CONGESTION_RELIEF_2):
        congestion_relief_2 = 1    
        
        
    return pd.Series(
        [active_transp_2, transit_2, bridge_2, street_2, freeway_2, infra_resiliency_er_2, congestion_relief_2], 
        index=['active_transp_2', 'transit_2', 'bridge_2', 'street_2', 
               'freeway_2', 'infra_resiliency_er_2', 'congestion_relief_2']
    )

In [130]:
"""
from Tiffany's new-metrics notebook
"""

ACTIVE_TRANSPORTATION_3 = ['bike', 'bicycle', 'cyclist', 
                         'pedestrian', 
                         ## including the spelling errors of `pedestrian`
                         'pedestrain',
                         'crosswalk', 
                         'bulb out', 'bulb-out', 
                         'active transp', 'traffic reduction', 
                         'speed reduction', 'ped', 'srts', 
                         'safe routes to school',
                         'sidewalk', 'side walk', 'Cl ', 'trail']
TRANSIT_3 = ['bus', 'metro', 'station', 
           'transit','fare', 'brt', 'yarts', 'railroad', 'highway-rail'] 
BRIDGE_3 = ["bridge", 'viaduct']
STREET_3 = ['traffic signal', 'resurface', 'resurfacing', 'slurry', 'seal' 
          'sign', 'stripe', 'striping', 'median', 
          'guard rail', 'guardrail', 
          'road', 'street', 
          'sinkhole', 'intersection', 'signal', 'curb',
          'light', 'tree', 'pavement', 'roundabout'
         ]

FREEWAY_3 = ['hov ', 'hot ', 'freeway', 'highway', 'express lanes', 'hwy']


INFRA_RESILIENCY_ER_3 = ['repair', 'emergency', 'replace','retrofit', 'er',
                       'rehab', 'improvements', 'seismic', 'reconstruct', 'restoration']

CONGESTION_RELIEF_3 = ['congestion', 'rideshare','ridesharing', 'vanpool', 'car share', 'demand management']

NOT_INC_3 = ['charging', 'fueling', 'cng', 'bridge', 'trail',
           'k-rail', 'guardrails', 'bridge rail', 'guard', 'guarrail']

def categorize_project_descriptions_3(row):
    """
    This function takes a individual type of work description (row of a dataframe)
    and returns a dummy flag of 1 if it finds keyword present in
    project categories (active transportation, transit, bridge, etc).
    A description can contain multiple keywords across categories.
    """
    # Make lowercase
    description = row.type_of_work.lower()
    
    # Store a bunch of columns that will be flagged
    # A project can involve multiple things...also, not sure what's in the descriptions
    active_transp_3 = 0
    transit_3 = 0
    bridge_3 = 0
    street_3 = 0
    freeway_3 = 0
    infra_resiliency_er_3 = 0
    congestion_relief_3 = 0
        
    if any(word in description for word in ACTIVE_TRANSPORTATION_3):
        active_transp_3 = 1
        
    if (any(word in description for word in TRANSIT_3) and 
        not any(exclude_word in description for exclude_word in NOT_INC_3)
       ):
        transit_3 = 1
        
    if any(word in description for word in BRIDGE_3):
        bridge_3 = 1
    if any(word in description for word in STREET_3):
        street_3 = 1
    if any(word in description for word in FREEWAY_3):
        freeway_3 = 1 
    if any(word in description for word in INFRA_RESILIENCY_ER_3):
        infra_resiliency_er_3 = 1
    if any(word in description for word in CONGESTION_RELIEF_3):
        congestion_relief_3 = 1    
        
        
    return pd.Series(
        [active_transp_3, transit_3, bridge_3, street_3, freeway_3, infra_resiliency_er_3, congestion_relief_3], 
        index=['active_transp_3', 'transit_3', 'bridge_3', 'street_3', 
               'freeway_3', 'infra_resiliency_er_3', 'congestion_relief_3']
    )

In [131]:
work_categories = df.apply(categorize_project_descriptions_2, axis=1)
df2 = pd.concat([df, work_categories], axis=1)
work_cols = list(work_categories.columns)
df2 = df2.assign(
    work_categories = df2[work_cols].sum(axis=1)
)

In [132]:
work_categories = df2.apply(categorize_project_descriptions_3, axis=1)
df3 = pd.concat([df2, work_categories], axis=1)
work_cols = list(work_categories.columns)
df3 = df3.assign(
    work_categories = df3[work_cols].sum(axis=1)
)

In [133]:
(
    df3
    >> filter(_.transit == 1)
    >> filter(_.transit_2 == 0)
    >> filter(_.transit_3 == 0)
    >> count(_.type_of_work)
    >> arrange(-_.n)
).head(30)

,type_of_work,n
3,Bridge Rail Replacement,7
4,Bridge Rail Replacement (tc),7
12,Install Guard Rail,5
36,Upgrade Guard Rail And End Treatments,5
6,Bridge Railing Replacement (tc),4
13,"Install Guard Rail, Signs And Markings.",4
2,Bridge Barrier Rail Replacement,3
10,"Guardrail. Upgrade The Existing Guardrails With New Guardrails, Transition Rails",3
14,Install Guard Rail/median Barrier And High-friction Surface Treatment.,3
17,"Install Intersection Lighting At 2 Intersections, Guard Railing And",3


In [134]:
transit = df3>>select(_.type_of_work, _.transit, _.transit_2, _.transit_3)

In [135]:
transit>>filter(_.transit_2==1)>>count(_.type_of_work)>>arrange(-_.n)

,type_of_work,n
124,"Public Outreach And Marketing For ""the Bus"" (tc)",9
188,Upgrade Traffic System Controller; Brt Phase( Tc),6
191,Yarts Public Outreach And Marketing (tc),6
7,Barrier Rail Replacement,5
21,"Const Ped,transit Facility And Traffic Improv",5
...,...,...
183,Translink Fare Collection System Fy 2010/2011 (tc),1
184,Two Highway Grad Crossings On The Metrolink San Bernadino Line,1
185,Union Station Master Plan:alameda Esplanade.,1
189,Walking School Bus Safety Prog,1


In [136]:
transit>>filter(_.transit_3==1)>>count(_.type_of_work)>>arrange(-_.n)

,type_of_work,n
105,"Public Outreach And Marketing For ""the Bus"" (tc)",9
162,Upgrade Traffic System Controller; Brt Phase( Tc),6
165,Yarts Public Outreach And Marketing (tc),6
19,"Const Ped,transit Facility And Traffic Improv",5
122,"Realign Roadway, Reduce Lanes And Transit-bike-pedestrian Lane (tc)",5
...,...,...
157,Translink Fare Collection System Fy 2010/2011 (tc),1
158,Two Highway Grad Crossings On The Metrolink San Bernadino Line,1
159,Union Station Master Plan:alameda Esplanade.,1
163,Walking School Bus Safety Prog,1


In [137]:
rail = (
    df3
    >> filter(_.transit_2==1)
    >> count(_.type_of_work)
    >>filter(_.type_of_work.str.contains('rail'))
    >> arrange(-_.n)
)

In [138]:
rail

,type_of_work,n
53,Gaurdrail And Safety Improvement,4
82,Install Traffic Control -railing And Plastic Sheeting For Eo Work.\n,3
59,Highway-rail Safety,2
150,Safety-railroad/highway Crossing Improvement. Improve Railroad Corridor By,2
60,"Highway-railroad Grade Crossing Hazard Elimination, With Match From State",1
61,"Highway-railroad Grade Crossing Safety Improvements And Hazard Elimination,",1
81,Install Traffic Control -railing And Plastic Sheeting For Eo Work.,1
112,"Pavement Resurfacing And Rehabilitation Including Bulb-outs, Metal Handrails,",1


In [139]:
# does railroad work? or do we need to also include rail?

In [140]:
(df>>filter(_.type_of_work.str.contains('rail'))>>count(_.type_of_work)>>select(_.type_of_work)).head(30)

,type_of_work
0,"""first Mile"" Bicycle And Pedestrian Trail Expansion (tc)"
1,Approximately 1.5 Miles Of Class 1 Trail South Of Historic Downtown Angles
2,Bicycle & Ped. Trail Improveme
3,Bicycle And Pedestrian Trail (tc)
4,Bicycle/pedetsrian Trail
5,Bike & Ped Trail
6,Bike & Ped. Trail
7,Bike & Pedestrian Trail
8,Bike And Ped Trail (tc)
9,Bike Lanes And Trail (tc)


In [141]:
(df>>filter(_.type_of_work.str.contains('rail'))>>count(_.type_of_work)>>select(_.type_of_work)).tail(60)

,type_of_work
242,Repair Slide On Roadway Which Caused Guardrail And Northbound Lane To Slip-
243,Replace Bridge Barrier Rails With New Guardrails
244,Replace Guardrail As Safety Funds Set-aside Project Allowed By Hsip Guidelines For
245,Replace Over 23000 Linear Feet Of Guardrail
246,Replace Single Beam Guardrailing With W-beam
247,Replace Substandard Guardrail
248,Resurface Bike Trail And Landscape Rest Stop
249,Roadside Safety Improvements Including Guardrail
250,Safety Improvement : Upgrade Guardrails And End Treatments
251,Safety-guardrail Replacement


In [142]:
## find transit phrases 

In [143]:
transit_1 = df3>>filter(_.transit==1)

In [144]:
s = transit_1.explode('type_of_work').groupby('type_of_work').size()

In [145]:
s

type_of_work
2020 Glass Fire -replace Guard Rail. Replace Wood Post With Metal Posts         2
: Transit/pedestrian Linkages,ped Improv,etc                                    1
: Transit/pedestrian Linkages,ped Improv,etc(tc)                                2
Abel Street Transit Connection                                                  1
Accuring Bus Wraps For Public Outreach(tc)                                      1
                                                                               ..
Widen Shoulders; Install Guard Rail                                             2
Yarts Public Outreach And Marketing (tc)                                        6
Yarts: Public Outreach And Marketing (yosemite Area Regional Transportation     2
Yosemite Area Regional Transportation System (yarts) Public Outreach And        2
Yosemite Area Regional Transportation System (yarts) Public Outreach And        2
Length: 218, dtype: int64

In [146]:
transit_2 = df3>>filter(_.transit_2==1)

In [147]:
s2= transit_2.explode('type_of_work').groupby('type_of_work').size()

In [148]:
transit_3 = df3>>filter(_.transit_3==1)

In [149]:
s3 = transit_3.explode('type_of_work').groupby('type_of_work').size()

### Using Regex

In [150]:
rail = (
    df3
    >> filter(_.transit_2==1)
    >> count(_.type_of_work)
    >>filter(_.type_of_work.str.contains('rail'))
    >> arrange(-_.n)
)

In [151]:
str_rail_1 = rail[['type_of_work']]
str_rail_1 = str_rail_1.squeeze().tolist() 
str_rail_1 = ' '.join(str_rail_1).lower()

In [152]:
str_rail_1

'gaurdrail and safety improvement install traffic control -railing and plastic sheeting for eo work.\n highway-rail safety safety-railroad/highway crossing improvement. improve railroad corridor by  highway-railroad grade crossing hazard elimination, with match from state  highway-railroad grade crossing safety improvements and hazard elimination,   install traffic control -railing and plastic sheeting for eo work. pavement resurfacing and rehabilitation including bulb-outs, metal handrails, '

In [153]:
 nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [154]:
# def clean_string(text):
#     text = re.sub(r"[^A-z\s]", "", text)
#     swords = [re.sub(r"[^A-z\s]", "", sword) for sword in stopwords.words('english')]
#     cleaned_list_of_words = [word for word in word_tokenize(text.lower()) if word not in swords]

#     return cleaned_list_of_words

In [155]:
# clean_str = clean_string(str_rail)
# clean_str

In [156]:
#from: https://github.com/UCLALuskinDataScience/UrbanDataScience/blob/main/weeks5-6/a.%20NLP%20Part%201.ipynb

def countWords(wordlist):
    
    swords = [re.sub(r"[^A-z\s]", "", sword) for sword in stopwords.words('english')]
    swords.append('tc')
    swords.append('&')
    swords.append('-')
    swords.append('1')
    swords.append(',')
            
    counts = {} 
    for word in wordlist:
        lword = word.lower()
        if lword in swords:
            continue
        elif lword in counts:
            counts[lword] +=1
        else:
            counts[lword] = 1

    # convert the dictionary to a dataframe
    # https://stackoverflow.com/questions/18837262/convert-python-dict-into-a-dataframe
    df = pd.DataFrame.from_dict(counts, orient='index', columns=['word_count'])

    # sort it by the word_count column
    df.sort_values('word_count', ascending=False, inplace=True)
    df.index.name = 'word'

    return df

In [157]:
str_rail_1 = re.sub(r"\s+", " ", str_rail_1)
wordlist_1 = str_rail_1.split()
words_1 = countWords(wordlist_1)

In [158]:
words_1.head(30)

,word_count
word,
safety,3
crossing,3
"elimination,",2
install,2
traffic,2
control,2
-railing,2
plastic,2
sheeting,2


In [159]:
## 

In [160]:
rail2 = (
    df3
    >> filter(_.transit_3==1)
    >> count(_.type_of_work)
    >>filter(_.type_of_work.str.contains('rail'))
    >> arrange(-_.n)
)

In [161]:
str_rail_3 = rail2[['type_of_work']]
str_rail_3 = str_rail_3.squeeze().tolist() 
str_rail_3 = ' '.join(str_rail_3).lower()

In [162]:
str_rail_3 = re.sub(r"\s+", " ", str_rail_3)
wordlist_3 = str_rail_3.split()
words_3 = countWords(wordlist_3)

In [163]:
words_3

,word_count
word,
crossing,3
safety,2
highway-railroad,2
grade,2
hazard,2
"elimination,",2
highway-rail,1
safety-railroad/highway,1
improvement.,1


In [164]:
df3>>filter(_.type_of_work.str.contains('Highway-rail'))>>count(_.type_of_work)

,type_of_work,n
0,Highway-rail Safety,2
1,"Highway-railroad Grade Crossing Hazard Elimination, With Match From State",1
2,"Highway-railroad Grade Crossing Safety Improvements And Hazard Elimination,",1


In [169]:
df3>>filter(_.type_of_work.str.contains('railroad/highway'))>>count(_.type_of_work)

,type_of_work,n
0,Safety-railroad/highway Crossing Improvement. Improve Railroad Corridor By,2


In [172]:
df3>>filter(_.type_of_work.str.contains('Highway/rail'))>>count(_.type_of_work)

,type_of_work,n


## Testing Transit (old)
- rail category was including trail 

In [55]:
(df2>>filter(_.transit==1)>>filter(_.transit_2==0)>>count(_.type_of_work)>>arrange(-_.n)).head(30)

,type_of_work,n
28,Class 1 Multi-use Trail (tc),8
88,Construct Class I Trail,8
118,Emergency Force Account Agreement With Apex Fence Co. For Guardrail Removal,8
182,"Install Temporary Traffic Signs, Remove And Replace Burned Guardrails And End",8
277,Upgrade Existing Guardrail.,7
285,Upgrade Guardrail,7
26,Class 1 Bike Trail,6
39,"Class I Bike Trail On Al Tahoe Adjacent To South Lake Tahoe Middle School, Bike",6
49,"Const Trailhead Impr, Park & Ride Parking Lot (tc)",6
83,Construct Class I Bikeway/trail,6


In [56]:
#k-rail is a concrete barrier used for road construction
len(df>>filter(_.type_of_work.str.contains('K-rail')))

43

In [57]:
print(f'When changing "rail" to " rail", transit obligations go down from',
      f'{len(df2>>filter(_.transit==1))} to {len(df2>>filter(_.transit_2==1))}')

When changing "rail" to " rail", transit obligations go down from 1119 to 431


In [58]:
print(f'When changing "rail" to " rail", District 4 Transit obligations go down from',
      f'{len(df2>>filter(_.dist==4)>>filter(_.transit==1))} to {len(df2>>filter(_.dist==4)>>filter(_.transit_2==1))}')

When changing "rail" to " rail", District 4 Transit obligations go down from 193 to 78


In [37]:
import re

In [38]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [39]:
df3= none[['type_of_work']]

In [40]:
df3

,type_of_work
13,Add One Mixed-flow Lane In Each Direction
51,Port Truck Reduction Program
55,Bluff Face Stabilization
66,Sf Bay Area: Regionwide: Implement Innovative Projects & Initiatives That
68,Ingrain Walking & Rolling Into School Culture (tc)
...,...
23449,"Install Its Adaptive System, Upgrade Detection And Synchronize Corridor. (tc)"
23521,Grade Separation
23539,Widen 2 To 4 Lanes - 2 Way Left Turn Lanes (tc)
23551,"Install Upgraded Led-enhanced Flashing Stop And Stop Ahead Signs, Install"


In [41]:
text = df3.squeeze().tolist() 

In [42]:
text

['Add One Mixed-flow Lane In Each Direction',
 'Port Truck Reduction Program',
 'Bluff Face Stabilization',
 'Sf Bay Area: Regionwide: Implement Innovative Projects & Initiatives That',
 'Ingrain Walking & Rolling Into School Culture (tc)',
 'Study On-campus Parking Pricing',
 'Sf Bay Area: Regionwide: Planning And Implementation Of Various Multi-modal',
 'Eliminate Curve; Install 4-way Stop; Install Eastbound Left Turn Lane And',
 'Install Dynamic Variable Speed Warning Signs At 2 Locations For Downhill',
 'Construction',
 '18/19 Traffic Count Program',
 'Scour Protection (tc)',
 'Grade Separation ,harney Lane At Upr.(tc)',
 'The Spare The Air Program Is An Ongoing Regional, Seasonal Public Outreach',
 'Install Traffic Signs, Flashing Beacons And Markings (tc)',
 'Landscaping & Enhancements - Phase 2',
 'Inst. Spd. Feedback Signs, Lts, Warning Sign(tc)',
 'Purchase Cng Refuse Truck',
 'Construct Left Turn Channelizations Along South Union Avenue.',
 'Ppm (rstp) Phase 3, 2019 (tc)',
 '

In [43]:
text = ' '.join(text).lower()

In [44]:
#swords = [re.sub(r"[^A-z\s]", "", sword) for sword in stopwords.words('english')]

In [45]:
def clean_string(text):
    text = re.sub(r"[^A-z\s]", "", text)
    swords = [re.sub(r"[^A-z\s]", "", sword) for sword in stopwords.words('english')]
    swords.append('tc')
    swords.append('install')
    
    cleaned_list_of_words = [word for word in word_tokenize(text.lower()) if word not in swords]

    return cleaned_list_of_words

In [46]:
clean_text = clean_string(text)

In [47]:
clean_text

['add',
 'one',
 'mixedflow',
 'lane',
 'direction',
 'port',
 'truck',
 'reduction',
 'program',
 'bluff',
 'face',
 'stabilization',
 'sf',
 'bay',
 'area',
 'regionwide',
 'implement',
 'innovative',
 'projects',
 'initiatives',
 'ingrain',
 'walking',
 'rolling',
 'school',
 'culture',
 'study',
 'oncampus',
 'parking',
 'pricing',
 'sf',
 'bay',
 'area',
 'regionwide',
 'planning',
 'implementation',
 'various',
 'multimodal',
 'eliminate',
 'curve',
 'way',
 'stop',
 'eastbound',
 'left',
 'turn',
 'lane',
 'dynamic',
 'variable',
 'speed',
 'warning',
 'signs',
 'locations',
 'downhill',
 'construction',
 'traffic',
 'count',
 'program',
 'scour',
 'protection',
 'grade',
 'separation',
 'harney',
 'lane',
 'uprtc',
 'spare',
 'air',
 'program',
 'ongoing',
 'regional',
 'seasonal',
 'public',
 'outreach',
 'traffic',
 'signs',
 'flashing',
 'beacons',
 'markings',
 'landscaping',
 'enhancements',
 'phase',
 'inst',
 'spd',
 'feedback',
 'signs',
 'lts',
 'warning',
 'signtc',
 

In [48]:
df4 = df3.squeeze()

In [49]:
df4 = df4.to_list()

In [50]:
df4

['Add One Mixed-flow Lane In Each Direction',
 'Port Truck Reduction Program',
 'Bluff Face Stabilization',
 'Sf Bay Area: Regionwide: Implement Innovative Projects & Initiatives That',
 'Ingrain Walking & Rolling Into School Culture (tc)',
 'Study On-campus Parking Pricing',
 'Sf Bay Area: Regionwide: Planning And Implementation Of Various Multi-modal',
 'Eliminate Curve; Install 4-way Stop; Install Eastbound Left Turn Lane And',
 'Install Dynamic Variable Speed Warning Signs At 2 Locations For Downhill',
 'Construction',
 '18/19 Traffic Count Program',
 'Scour Protection (tc)',
 'Grade Separation ,harney Lane At Upr.(tc)',
 'The Spare The Air Program Is An Ongoing Regional, Seasonal Public Outreach',
 'Install Traffic Signs, Flashing Beacons And Markings (tc)',
 'Landscaping & Enhancements - Phase 2',
 'Inst. Spd. Feedback Signs, Lts, Warning Sign(tc)',
 'Purchase Cng Refuse Truck',
 'Construct Left Turn Channelizations Along South Union Avenue.',
 'Ppm (rstp) Phase 3, 2019 (tc)',
 '

In [51]:
#need a cleaned list of strings, not a list of words

In [52]:
# cleaned_work_list = [clean_string(row)for row in df4] 

In [53]:
# cleaned_work_list

using the word clouds I added the following words to the categories: 
- trail
- restoration
- roundabout 

### CHANGING FOR ROUND 2
- change
  - "rail "
- adding:
   - 'demand management'

## To Function

In [19]:
# def add_categories(df):

#     ACTIVE_TRANSPORTATION = ['bike', 'bicycle', 'cyclist', 
#                              'pedestrian', 
#                              ## including the spelling errors of `pedestrian`
#                              'pedestrain',
#                              'crosswalk', 
#                              'bulb out', 'bulb-out', 
#                              'active transp', 'traffic reduction', 
#                              'speed reduction', 'ped', 'srts', 
#                              'safe routes to school',
#                              'sidewalk', 'side walk', 'Cl ', 'trail'
#                             ]
#     TRANSIT = ['bus', 'metro', 'station', #Station comes up a few times as a charging station and also as a train station
#                'transit','fare', 'brt', 'yarts', 'rail'
#                # , 'station' in description and 'charging station' not in description
#               ] 
#     BRIDGE = ["bridge", 'viaduct']
#     STREET = ['traffic signal', 'resurface', 'resurfacing', 'slurry', 'seal' 
#               'sign', 'stripe', 'striping', 'median', 
#               'guard rail', 'guardrail', 
#               'road', 'street', 
#               'sinkhole', 'intersection', 'signal', 'curb',
#               'light', 'tree', 'pavement', 'roundabout'
#              ]

#     FREEWAY = ['hov ', 'hot ', 'freeway', 'highway', 'express lanes', 'hwy']


#     INFRA_RESILIENCY_ER = ['repair', 'emergency', 'replace','retrofit', 'er',
#                            'rehab', 'improvements', 'seismic', 'reconstruct', 'restoration']

#     CONGESTION_RELIEF = ['congestion', 'rideshare','ridesharing', 'vanpool', 'car share']

#     NOT_INC = ['charging', 'fueling', 'cng']

#     def categorize_project_descriptions(row):
#         """
#         This function takes a individual type of work description (row of a dataframe)
#         and returns a dummy flag of 1 if it finds keyword present in
#         project categories (active transportation, transit, bridge, etc).
#         A description can contain multiple keywords across categories.
#         """
#         # Make lowercase
#         description = row.type_of_work.lower()
    
#         # Store a bunch of columns that will be flagged
#         # A project can involve multiple things...also, not sure what's in the descriptions
#         active_transp = 0
#         transit = 0
#         bridge = 0
#         street = 0
#         freeway = 0
#         infra_resiliency_er = 0
#         congestion_relief = 0
        
#         if any(word in description for word in ACTIVE_TRANSPORTATION):
#             active_transp = 1
        
#         #if any(word in description if instanceof(word, str) else word(description) for word in TRANSIT)

#         if (any(word in description for word in TRANSIT) and 
#             not any(exclude_word in description for exclude_word in NOT_INC)
#            ):
#             transit = 1
        
#         if any(word in description for word in BRIDGE):
#             bridge = 1
#         if any(word in description for word in STREET):
#             street = 1
#         if any(word in description for word in FREEWAY):
#             freeway = 1 
#         if any(word in description for word in INFRA_RESILIENCY_ER):
#             infra_resiliency_er = 1
#         if any(word in description for word in CONGESTION_RELIEF):
#             congestion_relief = 1    
        
        
#         return pd.Series(
#             [active_transp, transit, bridge, street, freeway, infra_resiliency_er, congestion_relief], 
#             index=['active_transp', 'transit', 'bridge', 'street', 
#                    'freeway', 'infra_resiliency_er', 'congestion_relief']
#         )
    
    
#     work_categories = df.apply(categorize_project_descriptions, axis=1)
    
#     work_cols = list(work_categories.columns)
#     df2 = pd.concat([df, work_categories], axis=1)
    
#     df2 = df2.assign(work_categories = df2[work_cols].sum(axis=1))
    
#     return(df2)




In [20]:
# test = add_categories(df)

In [21]:
# test